# Recoleccion de datos a partir de Sophia

## 0. Librerías útiles

- Elasticsearch es el motor de búsqueda utilizado por Sophia
- Requiere la instalación del paquete elasticsearch

In [1]:
import elasticsearch
from datetime import datetime
import spacy
from spacy.matcher import PhraseMatcher 
# Pandas
import pandas
from pandasql import sqldf
# Geopandas
import geopandas as gpd

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/maquinitaz/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
elasticsearch.__version__
nlp = spacy.load("es_core_news_md")

## 1. Parámetros de la búsqueda

In [3]:
country="chile"
from_="2021-09-01"
to_="2021-12-31"
media_outlets=["elllanquihue","elaustral","laestrelladechiloe","elheraldoaustral","radiosago",
               "elrepuertero","elvacanudo","elhuemul","seminariolocal","elquellonino","elinsular",
               "radiopudeto","radioacogida","elcalbucano","segundos33",
               "prensadelestuario","fresiaahora","soychiloe"]

## 2. Parámetros de conexión al motor de búsqueda de Sophia

In [4]:
IP = "search.sophia2.org"
PORT = 9200
USER= "elastic"
PASS= "AxahEIWKsagna321"

In [5]:
#NO MODIFICAR

es = elasticsearch.Elasticsearch(
    IP,
    #port=PORT,
    http_auth=(USER, PASS)
)

## 3. Búsqueda de noticias de prensa

- Preparación de la consulta en el formato solicitado por Elasticsearch

In [6]:
#NO MODIFICAR

query = { 
    "bool": {
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }}},
        { "term":  { "country": country }},
        { "terms":  { "media_outlet": media_outlets }} 
    ]
    }  
}

- Envio de la consulta
- No se puede devolver más de 10.000 noticias. En caso de obtener 10.000 respuestas, se recomienda buscar en un periodo de tiempo más corto o modificar la consulta.

In [7]:
res = es.search(index="news", query=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 10000 noticias encontradas...


## 4. Guardar los datos en un dataframe Pandas y en un archivo CSV

In [8]:
import pandas as pd

data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[]}

df = pd.DataFrame(data)  
  
for hit in res['hits']['hits']:
    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    
    new_row = {'id_news':id_news, 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date}
    
    df = df.append(new_row, ignore_index=True)

df

/tmp/ipykernel_3546/3725804544.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_3546/3725804544.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_3546/3725804544.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_3546/3725804544.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_3546/3725804544.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

,id_news,country,media_outlet,url,title,text,date
0,21887365.0,chile,radiosago,https://www.radiosago.cl/lenta-habilitacion-de...,Lenta habilitación de mesas en algunos locales...,Un total de 2.127 mesas receptoras de sufragio...,2021-12-19
1,21887883.0,chile,radiosago,https://www.radiosago.cl/pdi-detiene-a-sujeto-...,PDI detiene a sujeto por homicidio frustrado y...,Detectives de la Brigada de Homicidios Puerto ...,2021-11-12
2,21887884.0,chile,radiosago,https://www.radiosago.cl/superintendencia-de-s...,Superintendencia de Salud reconoció a la Unida...,La Superintendencia de Salud hizo entrega de u...,2021-11-11
3,21887888.0,chile,radiosago,https://www.radiosago.cl/aumentan-toma-de-pcr-...,Aumentan toma de PCR y vacunación se extiende ...,Debido al aumento de casos por COVID 19 y para...,2021-11-11
4,21887412.0,chile,radiosago,https://www.radiosago.cl/camara-de-comercio-de...,Cámara de Comercio de Puerto Montt manifestó m...,A través de una carta dirigida al alcalde Gerv...,2021-12-17
...,...,...,...,...,...,...,...
9995,47532472.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,Chiloé ya vive la Jornada por la Rehabilitació...,"Con gran entusiasmo y participación, ayer pers...",2021-11-27
9996,47533268.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,Marcianeke es el artista nacional más escuchad...,El año que comienza a concluir tiene un fenóme...,2021-12-04
9997,47532767.0,chile,elaustral,https://www.australosorno.cl/impresa/2021/12/2...,Joe Biden endurece los estándares de emisiones...,"El Gobierno del presidente estadounidense, Joe...",2021-12-21
9998,47533270.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,Serrat le dirá adiós a los escenarios e incluy...,"Después de casi 60 años de carrera, el músico ...",2021-12-04


- Verifiquemos si existen noticias con la misma URL

In [9]:
df['url'].duplicated().any()

True

- Eliminar las noticias duplicadas

In [10]:
df = df.drop_duplicates(subset='url', keep='first')

In [11]:
df = df.reset_index(drop=True)
df

,id_news,country,media_outlet,url,title,text,date
0,21887365.0,chile,radiosago,https://www.radiosago.cl/lenta-habilitacion-de...,Lenta habilitación de mesas en algunos locales...,Un total de 2.127 mesas receptoras de sufragio...,2021-12-19
1,21887883.0,chile,radiosago,https://www.radiosago.cl/pdi-detiene-a-sujeto-...,PDI detiene a sujeto por homicidio frustrado y...,Detectives de la Brigada de Homicidios Puerto ...,2021-11-12
2,21887884.0,chile,radiosago,https://www.radiosago.cl/superintendencia-de-s...,Superintendencia de Salud reconoció a la Unida...,La Superintendencia de Salud hizo entrega de u...,2021-11-11
3,21887888.0,chile,radiosago,https://www.radiosago.cl/aumentan-toma-de-pcr-...,Aumentan toma de PCR y vacunación se extiende ...,Debido al aumento de casos por COVID 19 y para...,2021-11-11
4,21887412.0,chile,radiosago,https://www.radiosago.cl/camara-de-comercio-de...,Cámara de Comercio de Puerto Montt manifestó m...,A través de una carta dirigida al alcalde Gerv...,2021-12-17
...,...,...,...,...,...,...,...
4252,47533619.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,columna de opinión lenguaje,Durante este tiempo de pandemia hemos visto qu...,2021-11-23
4253,47532753.0,chile,elaustral,https://www.australosorno.cl/impresa/2021/12/2...,El docente y ambientalista Ricardo Becerra pub...,"""Osorno, imágenes con Historia"" se llama el pr...",2021-12-21
4254,47532755.0,chile,elaustral,https://www.australosorno.cl/impresa/2021/12/2...,"Mercados caen tras el balotaje, pero ven efect...",El mercado local reaccionó este lunes ante el ...,2021-12-21
4255,47532756.0,chile,elaustral,https://www.australosorno.cl/impresa/2021/12/2...,Informe confirma cinco nuevas muertes por el c...,Cinco nuevos fallecidos contagiados con covid ...,2021-12-21


- Encontrar las comunas

In [12]:
territory = gpd.read_file(r'./shape/comunas.shp', encoding='utf-8')
territory=territory[territory["codregion"] == 10]

comunas = territory.Comuna.values.tolist()
comunas

['San Pablo',
 'Puqueldón',
 'Fresia',
 'Llanquihue',
 'Osorno',
 'Purranque',
 'Puyehue',
 'Cochamó',
 'Ancud',
 'Quellón',
 'Queilén',
 'Chonchi',
 'Puerto Montt',
 'Río Negro',
 'Castro',
 'Dalcahue',
 'Quemchi',
 'San Juan de la Costa',
 'Calbuco',
 'Chaitén',
 'Los Muermos',
 'Maullín',
 'Quinchao',
 'Curaco de Vélez',
 'Puerto Octay',
 'Frutillar',
 'Puerto Varas',
 'Hualaihué',
 'Palena',
 'Futaleufú']

In [13]:
matcher_comunas = PhraseMatcher(nlp.vocab)
for comuna in comunas:
    matcher_comunas.add(comuna, [nlp(comuna)])

In [14]:
for index,row in df.iterrows():
    txt = row["text"]
    
    try:
    
        doc = nlp(txt)
        matches_comunas = matcher_comunas(doc)

        for match_id, start, end in matches_comunas:
            span = doc[start:end]  # The matched span
    
    except:
        pass

In [15]:
for index,row in df.iterrows():
    txt = row["text"]
    
    try:
    
        doc = nlp(txt)
        matches_comunas = matcher_comunas(doc)

        for match_id, start, end in matches_comunas:
            span = doc[start:end]  # The matched span
            df.at[index,span.text.replace(" ","_")]=1
    except Exception as e:
        print(e)
        pass

- Guardar los archivos en un CSV

In [16]:
nombre_archivo=country+"_"+from_+"_"+to_+".csv"
print("El archivo se llama:")
print(nombre_archivo)
df.to_csv("./"+nombre_archivo)

El archivo se llama:
chile_2021-09-01_2021-12-31.csv
